In [1]:
# STOCK DATA
# ~~~~~~~~~~~

# libaries
import pandas as pd
import yfinance as yf
import numpy as np
from datetime import timedelta
import time
import gc
from tqdm import tqdm

# pandas read.html to gather stock tickers from S&P500 Wiki page
def sp500_tickers():
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    try:
        table = pd.read_html(url)[0]
        tickers = table["Symbol"].tolist()
        print(f"{len(tickers)} tickers gathered from S&P500 Wiki page.")
        
        return tickers
    # error msg    
    except Exception as e:
        print(f"ERROR: {e}")
        
        return []

In [2]:
# Using yfinance to gather historical stock market data over 10 years for each ticker
def get_stock_data(tickers):
    try:
        stock_data = yf.download(tickers, period="10y", interval="1d", group_by="ticker", progress=False)
        print(f"Data gathered for all stocks: {len(tickers)}.")
        
        return stock_data
    # error msg    
    except Exception as e:
        print(f"ERROR for: {tickers}: {e}")
        
        return None

In [3]:
# Using stock_data to create a full historical dataframe all_stock_data, and copy it to avoid SettingWithCopyWarning
def process_data(stock_data, tickers):
    all_stock_data = []
    
    for ticker in tickers:
        
        if ticker in stock_data:
            ticker_data = stock_data[ticker].copy()  
            # add "Return", "Volatility", "Ticker", and drop nulls
            ticker_data.loc[:, "Return"] = ticker_data["Close"].pct_change()
            ticker_data.loc[:, "Volatility"] = ticker_data["Return"].rolling(window=30).std()
            ticker_data.loc[:, "Ticker"] = ticker
            ticker_data = ticker_data.dropna().reset_index()  
            all_stock_data.append(ticker_data)
            
    return pd.concat(all_stock_data, ignore_index=True)

In [4]:
# Collecting data for each ticker in all_stock_data
# avoiding crash/timeout using batches
def batch_collect(tickers, batch_size=50):
    all_stock_data = []
    
    for i in tqdm(range(0, len(tickers), batch_size), desc="Fetching data..."):
        batch_tickers = tickers[i:i + batch_size]
        stock_data = get_stock_data(batch_tickers)
        
        if stock_data is not None:
            all_stock_data.append(process_data(stock_data, batch_tickers))
        gc.collect() # clear memory using garbage collection (gc)

    return pd.concat(all_stock_data, ignore_index=True)

In [5]:
# Gather S&P 500 tickers and save the full data (df_stocks is the base dataframe, saved as full_stock_data.csv)
tickers = sp500_tickers()
df_stocks = batch_collect(tickers, batch_size=50)

df_stocks.to_csv("full_stock_data.csv", index=False)
print(f"Stock data records: {df_stocks.shape[0]} saved.")

503 tickers gathered from S&P500 Wiki page.


Fetching data...:   0%|          | 0/11 [00:00<?, ?it/s]

YF.download() has changed argument auto_adjust default to True


Fetching data...:   9%|▉         | 1/11 [00:02<00:21,  2.16s/it]

Data gathered for all stocks: 50.


...

Fetching data...:  91%|█████████ | 10/11 [00:18<00:01,  1.68s/it]

Data gathered for all stocks: 50.


Fetching data...: 100%|██████████| 11/11 [00:18<00:00,  1.73s/it]

Data gathered for all stocks: 3.


Stock data records: 1213060 saved.


In [11]:
# Using EWMA volatility with 0.94 lambda and a 0.75 quantile thresholh to capture 30-day windows of stock analysis
def EWMA(df, lambda_=0.94, threshold_quantile=0.75):
    df['EWMA_Volatility'] = df['Return'].ewm(span=30, adjust=False).std()
    
    volatility_threshold = df['EWMA_Volatility'].quantile(threshold_quantile)
    print(f"Volatility Threshold Quantile {threshold_quantile}, Volatility Threshold: {volatility_threshold}")
    filtered_df = df[df['EWMA_Volatility'] >= volatility_threshold]
    
    return filtered_df

# applying EWMA function to df_stocks and saving the filtered data as df_stockhistory
df_stockhistory = EWMA(df_stocks)

# show first and last few rows
print("Preview of the first few rows of the filtered DataFrame based on EWMA volatility threshold:")
print(df_stockhistory.head())
print("\nPreview of the last few rows of the filtered DataFrame:")
print(df_stockhistory.tail())

Volatility Threshold Quantile 0.75, Volatility Threshold: 0.020819494379046247
Preview of the first few rows of the filtered DataFrame based on EWMA volatility threshold:
Price       Date        Open        High         Low       Close      Volume  \
750   2018-04-24  133.066493  133.066493  126.873912  129.606125  13958994.0   
751   2018-04-25  129.586800  130.005649  126.951240  128.130478   7421658.0   
752   2018-04-26  128.027407  128.639576  126.409988  127.002823   4932663.0   
921   2018-12-27  120.670680  125.405441  120.262959  125.405441   3359564.0   
926   2019-01-04  122.807950  126.247230  122.334463  125.813202   3582140.0   

Price    Return  Volatility Ticker  Adj Close  EWMA_Volatility  
750   -0.068325    0.020710    MMM        NaN         0.022209  
751   -0.011386    0.020726    MMM        NaN         0.021525  
752   -0.008801    0.020679    MMM        NaN         0.020829  
921    0.023837    0.019292    MMM        NaN         0.020960  
926    0.041140    0.02

In [13]:
df_stockhistory.shape

(303265, 11)

In [15]:
# Dropping "Adj Close" variable because we collected no data, no reason to suspect we need it for our analysis
df_stockhistory.drop(columns=['Adj Close'])

Price         Date        Open        High         Low       Close  \
750     2018-04-24  133.066493  133.066493  126.873912  129.606125   
751     2018-04-25  129.586800  130.005649  126.951240  128.130478   
752     2018-04-26  128.027407  128.639576  126.409988  127.002823   
921     2018-12-27  120.670680  125.405441  120.262959  125.405441   
926     2019-01-04  122.807950  126.247230  122.334463  125.813202   
...            ...         ...         ...         ...         ...   
1212845 2024-05-08  165.788278  166.205029  159.755282  163.644974   
1212846 2024-05-09  164.518166  167.088139  163.644965  166.830154   
1212847 2024-05-10  167.405688  168.675791  166.988922  167.733124   
1212848 2024-05-13  168.120084  170.233622  166.502697  166.611847   
1212850 2024-05-15  169.529117  173.637109  169.320734  171.940323   

Price        Volume    Return  Volatility Ticker  EWMA_Volatility  
750      13958994.0 -0.068325    0.020710    MMM         0.022209  
751       7421658.0 -0.011386    0.020726    MMM         0.021525  
752       4932663.0 -0.008801    0.020679    MMM         0.020829  
921       3359564.0  0.023837    0.019292    MMM         0.020960  
926       3582140.0  0.041140    0.020327    MMM         0.022849  
...             ...       ...         ...    ...              ...  
1212845   4673100.0 -0.020956    0.023770    ZTS         0.023062  
1212846   2145300.0  0.019464    0.023831    ZTS         0.022756  
1212847   2010700.0  0.005413    0.023839    ZTS         0.022021  
1212848   2125800.0 -0.006685    0.023746    ZTS         0.021430  
1212850   2585700.0  0.026115    0.023927    ZTS         0.020923  

[303265 rows x 10 columns]

In [21]:
# Save data to CSV with UTF-8 encoding (otherwise receive errors)
df_stockhistory.to_csv("df_stockhistory.csv", index=False, encoding='utf-8-sig')

print("Data saved as: 'df_stockhistory.csv'")

Data saved as: 'df_stockhistory.csv'


Added stock "Name" variable using an online source and mapping each ticker to their correponding name in separate code.